In [12]:
import os
import json
from datetime import datetime
import pandas as pd

In [13]:
# modificar segun entorno local
os.chdir( "/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"
folder_gmaps = "gmaps"

In [14]:
# extraemos nombres de cada estado segun nombre de carpeta
state_name = pd.DataFrame(
    pd.DataFrame(
        os.listdir(
            os.path.join(folder_data,folder_gmaps))).rename(columns={0:"state"})["state"].str.split("-").str[1])
state_name.head(15)

,state
0,Indiana
1,Iowa
2,Kansas
3,Kentucky
4,Louisiana


In [15]:
# valor piloto;
# en el loop seria for i in range(min(state_name.index),max(state_name.index))
i = 1

# creamos una variable con el nombre de estado
# para facilitar el loop para todos los estados
state = state_name["state"][i]
folder_state = "review-" + str(state)

In [16]:
# contamos numero de archivos al interior de cada carpeta del estado
# para facilitar el loop
count_file = 0
# Iterate
for path in os.listdir(os.path.join(folder_data,folder_gmaps,folder_state)):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder_data,folder_gmaps,folder_state,path)):
        count_file += 1
count_file

16

In [17]:
# extraemos la informacion
data = []
for f in range(1, count_file):
    with open(
        os.path.join(folder_data,folder_gmaps,folder_state, str(f) + ".json"), "r") as file:
        for line in file:
            data.append(json.loads(line))
gmaps_state = pd.DataFrame(data)
gmaps_state.info()
gmaps_state.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 137.3+ MB


,user_id,name,time,rating,text,pics,resp,gmap_id
0,116210082545550472083,Brian A,1528719383671,5,Every time I have been to this place I am in a...,None,None,0x89b7a76096bdad0b:0x66e4be0af3dc1720
1,108990491488903625674,Marsha J,1556106162554,5,"Good pharmacy and fast service, majority of th...",None,"{'time': 1556236566548, 'text': 'Hi Marsha, th...",0x89b7a76096bdad0b:0x66e4be0af3dc1720


In [18]:
# indentificamos las columnas de interes y aquellas que se deben desanidar
vars_interes=[
    "gmap_id" # pk google maps
    ,"user_id"
    ,"name"
    ,"time"
    ,"rating"
    ,"text"
    #,"pics" # irrelevante
    #,"resp" # desanidar
    ]

vars_desanidar=[
    "gmap_id" # pk google maps
    ,"user_id" # pk user >> resp
    ,"resp" # desanidar
    ]


In [19]:
# nos concentramos en: vars_desanidar
gmaps_state_dsndr = pd.DataFrame(gmaps_state[vars_desanidar]).dropna(subset=["gmap_id","user_id"])
gmaps_state_dsndr.info()
gmaps_state_dsndr.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   gmap_id  object
 1   user_id  object
 2   resp     object
dtypes: object(3)
memory usage: 51.5+ MB


,gmap_id,user_id,resp
0,0x89b7a76096bdad0b:0x66e4be0af3dc1720,116210082545550472083,None
1,0x89b7a76096bdad0b:0x66e4be0af3dc1720,108990491488903625674,"{'time': 1556236566548, 'text': 'Hi Marsha, th..."


In [20]:
# se normaliza la columna
gmaps_state_dsndr = gmaps_state_dsndr.set_index(["gmap_id","user_id"])
gmaps_state_dsndda = pd.json_normalize(
    gmaps_state_dsndr["resp"]).set_index(gmaps_state_dsndr.index)
gmaps_state_dsndda.reset_index(inplace=True) # mueve le indice a una columna
# renombramos para facilitar la union mas abajo
gmaps_state_dsndda.rename(
    columns={"time":"resp_time","text":"resp_text"},
    inplace=True
    )
gmaps_state_dsndda.info()
gmaps_state_dsndda.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   gmap_id    object 
 1   user_id    object 
 2   resp_time  float64
 3   resp_text  object 
dtypes: float64(1), object(3)
memory usage: 68.7+ MB


,gmap_id,user_id,resp_time,resp_text
0,0x89b7a76096bdad0b:0x66e4be0af3dc1720,116210082545550472083,NaN,NaN
1,0x89b7a76096bdad0b:0x66e4be0af3dc1720,108990491488903625674,1.556237e+12,"Hi Marsha, thank you for your feedback. We kno..."


In [21]:
# solo incluimos las variables de interes
gmaps_state_interes = gmaps_state[vars_interes].copy().dropna(subset=["gmap_id","user_id"])
# renombramos para facilitar la union mas abajo
gmaps_state_interes.rename(
    columns={"name":"user_name","time":"user_time","text":"user_text"},
    inplace=True
    )
gmaps_state_interes.info()
gmaps_state_interes.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250000 entries, 0 to 2249999
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   gmap_id    object
 1   user_id    object
 2   user_name  object
 3   user_time  int64 
 4   rating     int64 
 5   user_text  object
dtypes: int64(2), object(4)
memory usage: 103.0+ MB


,gmap_id,user_id,user_name,user_time,rating,user_text
0,0x89b7a76096bdad0b:0x66e4be0af3dc1720,116210082545550472083,Brian A,1528719383671,5,Every time I have been to this place I am in a...
1,0x89b7a76096bdad0b:0x66e4be0af3dc1720,108990491488903625674,Marsha J,1556106162554,5,"Good pharmacy and fast service, majority of th..."


In [22]:
# unir los dataframes
gmaps_state_norm = pd.merge(
    gmaps_state_interes,
    gmaps_state_dsndda,
    on=["gmap_id","user_id"]
    ,how="left"
    )

# generamos una columna state
# para facilitar la union de todos los datos
gmaps_state_norm["state"] = state

# movemos 'state' a la primera fila para facilitar la visualizacion
first_column = gmaps_state_norm.pop("state")
gmaps_state_norm.insert(0, "state", first_column)

# Convierte la columna 'user_time' a datetime y almacena el resultado en una nueva columna 'time_total'
gmaps_state_norm['user_time_total'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms',errors='coerce')
gmaps_state_norm['user_time_year'] = gmaps_state_norm['time_total'].dt.year
gmaps_state_norm['user_time_month'] = gmaps_state_norm['time_total'].dt.month
gmaps_state_norm['user_time_day'] = gmaps_state_norm['time_total'].dt.day
gmaps_state_norm['user_time_hour'] = gmaps_state_norm['time_total'].dt.hour
gmaps_state_norm['resp_time'] = pd.to_datetime(gmaps_state_norm['resp_time'], errors='coerce')

# df final por estado
gmaps_state_norm.to_parquet(os.path.join(
    folder_pipeline, str(folder_state) + "_norm.parquet"))
gmaps_state_norm.info()
print(gmaps_state_norm.isna().sum())
gmaps_state_norm.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2305522 entries, 0 to 2305521
Data columns (total 9 columns):
 #   Column     Dtype  
---  ------     -----  
 0   state      object 
 1   gmap_id    object 
 2   user_id    object 
 3   user_name  object 
 4   user_time  int64  
 5   rating     int64  
 6   user_text  object 
 7   resp_time  float64
 8   resp_text  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 158.3+ MB
state              0
gmap_id            0
user_id            0
user_name          0
user_time          0
rating             0
user_text    1006455
resp_time    2007989
resp_text    2007989
dtype: int64


,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text
0,Maryland,0x89b7a76096bdad0b:0x66e4be0af3dc1720,116210082545550472083,Brian A,1528719383671,5,Every time I have been to this place I am in a...,NaN,NaN
1,Maryland,0x89b7a76096bdad0b:0x66e4be0af3dc1720,116210082545550472083,Brian A,1528719383671,5,Every time I have been to this place I am in a...,NaN,NaN
